In [2]:
import sys; sys.path.append('..'); from wwa import *

xn, xx, yn, yx = [-67,-47,-27,-7]
mapproj = cartopy.crs.PlateCarree()

# ERA5 FWI download via API

In [ ]:
import cdsapi

c = cdsapi.Client()

for y in range(1950,2023):
    
    c.retrieve(
        'cems-fire-historical-v1',
        {
            'product_type': 'reanalysis',
            'variable': [
                'build_up_index', 'drought_code', 'duff_moisture_code',
                'fine_fuel_moisture_code', 'initial_fire_spread_index',
                'keetch_byram_drought_index',
            ],
            'dataset_type': 'consolidated_dataset',
            'system_version': '4_1',
            'year': str(y),
            'month': [
                '01', '02', '03',
                '04', '05', '06',
                '07', '08', '09',
                '10', '11', '12',
            ],
            'day': [
                '01', '02', '03',
                '04', '05', '06',
                '07', '08', '09',
                '10', '11', '12',
                '13', '14', '15',
                '16', '17', '18',
                '19', '20', '21',
                '22', '23', '24',
                '25', '26', '27',
                '28', '29', '30',
                '31',
            ],
            'grid': '0.25/0.25',
            'format': 'netcdf',
            'area': [
                -12, -62, -22,
                -52,
            ],
        },
        'data/era5-fire/fwi-subindices_'+str(y)+'.nc')

# Get DSR

In [5]:
# Compile FWI data into a single file
fl = sorted(glob.glob("../../00_WWA_project_folder/live/data/era5_fire/*consolidated*.nc"))

fwi_cns = [wrap_lon(xr.open_dataset(fnm)).fwinx.sel(longitude = slice(xn, xx), latitude = slice(yn, yx)).reset_coords(drop = True) for fnm in fl]

int_start = (fwi_cns[-1].time[-1]+1)

fwi_int = wrap_lon(xr.open_dataset("../../00_WWA_project_folder/live/data/era5_fire/era5_025_fwi_intermediate.nc"))
fwi_int = fwi_int.fwinx.sel(longitude = slice(xn, xx), latitude = slice(yn, yx), time = slice(int_start, None)).reset_coords(drop = True)

fwi = xr.concat(fwi_cns + [fwi_int], "time")
fwi = fwi.assign_attrs(note = "Intermediate data after "+str(fwi_int.time.min().values)[:10])

fwi.to_netcdf("data/era5_fwi_"+"_".join([str(d) for d in [xn,xx,yn,yx]])+".nc")

In [10]:
dsr = (0.0272 * (fwi ** 1.71)).rename("dsr")

In [12]:
dsr.to_netcdf("data/era5_dsr.nc")